SEMAINE EN LABO 1: 10/11

Objectifs= 
- valeurs de shapley
- tests d'indépendance: hsic, fast rank test, matrices de similarité, shapley

Valeurs de Shapley: 2^N=P(N)
Jeux coopératif N: N ensemble des joueurs. V: 2^N-> R*
Valeur de shapley: parmi la grande coalition, phi:N->R, impact du joueur dans la coalition.

4 axiomes:
- i acteur simple. pour tout S dans 2^N, is phi(S ou {i})=v(S), alors phi(i)=0

